In [1]:
using QuadraticTeamDecisionSolver, Distributions, Statistics, LinearAlgebra, Plots, LaTeXStrings, BlockDiagonals, Plots.PlotMeasures

In [2]:
dpi=1000;
l=1.5;

In [3]:
N = K = 2;
L = 3; #multiple antennas -> vector valued gammas
P = 0.7; 
σₕ = 1.0;
σₙ = σₕ/10;
ϵ = 0.2;
p = QuadTeamProblem(N, N * K * L * ones(Int64, N), L*ones(Int64, N), ComplexF64);
t = teamMMSEproblem(
	P,
	N,
	L,
	K,
	σₕ * ones(Float64, N),
	σₙ * ones(Float64, N),
	ϵ * ones(Float64, N)
);


In [4]:
S = 100;
iters = 10;
Y_data, _, _ = generateTeamMMSEsamples(t, S);
_, Q_data, R_data = generateTeamMMSEsamples(t, Int(ceil(S/2)));

In [5]:
C1 = mean(Q_data[1, 1]);
C2 = mean(Q_data[2, 2]);
h1 = h2 = 0.5;
λ1 = λ2 = σₙ^2; #noise variance

K1(x, t) = matrixExponentialKernel([h1], [C1], [λ1], x, t);
K2(x, t) = matrixExponentialKernel([h2], [C2], [λ2], x, t);
kernels = [K1, K2];
λ = [λ1, λ2];


In [6]:
@time Y, Q, R = reformatData(Y_data, Q_data, R_data);

  0.680515 seconds (1.20 M allocations: 74.382 MiB, 1.96% gc time, 98.69% compilation time)


In [7]:
@time G_o, G_p, C, Q_f = solverPreprocessing(Y, Q, λ, kernels, L, K, N, S);

  1.524529 seconds (2.82 M allocations: 190.026 MiB, 3.80% gc time, 98.06% compilation time)


In [8]:
@time g = optimizedGaussSeidel(G_p, C, Q_f, R, iterations=iters);

MethodError: MethodError: no method matching optimizedGaussSeidel(::Vector{Matrix{ComplexF64}}, ::Vector{Matrix{ComplexF64}}, ::Vector{Cholesky{ComplexF64, Matrix{ComplexF64}}}, ::Matrix{AbstractMatrix{ComplexF64}}, ::Vector{Vector{ComplexF64}}; iterations::Int64)

Closest candidates are:
  optimizedGaussSeidel(::Vector{<:Matrix}, ::Vector, !Matched::Matrix{<:AbstractMatrix}, !Matched::Vector{<:Vector}; iterations)
   @ QuadraticTeamDecisionSolver ~/QuadraticTeamDecisionSolver.jl/src/fixedPointSolvers.jl:282


In [9]:
γ = vcat.(g...);
conv = norm.(diff(γ));


UndefVarError: UndefVarError: `g` not defined

In [10]:
γ_f = [[v[l:l+L-1] for l in 1:L:length(v)] for v in γ];


UndefVarError: UndefVarError: `γ` not defined

In [11]:
variance = norm.(diff(cov.(γ_f)));
first_moment = norm.(diff(mean.(γ_f)));

UndefVarError: UndefVarError: `γ_f` not defined

In [12]:
arg = plot(
	1:iters,
	conv,
	label = L"||\gamma_{k+1}(\mathbf{Y})-\gamma_k(\mathbf{Y})||",
	dpi = dpi,
	linewidth = l,
	yscale = :log10,
	title = "Convergence in Argument",
	xlabel = "Iterations " * L"k",
)
meas = plot(
	[variance, first_moment],
	yscale = :log10,
	label = [L"|\mathbb{E}[\gamma_{k+1}(\mathbf{Y})] - \mathbb{E}[\gamma_{k}(\mathbf{Y})]|" L"|\mathrm{Var}[\gamma_{k+1}(\mathbf{Y})] - \mathrm{Var}[\gamma_{k}(\mathbf{Y})]|"],
	dpi = dpi,
	linewidth = l,
	title = "Convergence in Measure",
	xlabel = "Iterations " * L"k",
)
plot(
	arg,
	meas,
	layout = (1, 2),
	size = (1000, 300),
	left_margin = 20px,
	bottom_margin = 20px,
)


UndefVarError: UndefVarError: `conv` not defined